# Implementation

In [1]:
# Importar Libraries
library(tidyverse)
library(rminer)
library(forecast)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'rminer' was built under R version 3.6.3"Warning message:
"package 'forecast' was built under R version 3.6.3"Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


## Import Time Series

In [2]:
ts1 <- read.csv(file = './cenarios/Cenario 1/TS1.csv')
ts2 <- read.csv(file = './cenarios/Cenario 1/TS2.csv')
ts3 <- read.csv(file = './cenarios/Cenario 1/TS3.csv')
ts4 <- read.csv(file = './cenarios/Cenario 1/TS4.csv')
ts5 <- read.csv(file = './cenarios/Cenario 1/TS5.csv')

ts1 = ts1 %>% select(-X)
ts2 = ts2 %>% select(-X)
ts3 = ts3 %>% select(-X)
ts4 = ts4 %>% select(-X)
ts5 = ts5 %>% select(-X)

head(ts1)
head(ts2)
head(ts3)
head(ts4)
head(ts5)

all,weather,maxtemp,RH,maxwind
2332,0,13,87,0
2801,0,14,94,45
2375,0,14,82,55
3447,1,13,78,0
4823,0,16,81,37
4978,0,16,73,0


female,weather,maxtemp,RH,maxwind
1115,0,13,87,0
1217,0,14,94,45
1168,0,14,82,55
1617,1,13,78,0
2469,0,16,81,37
2564,0,16,73,0


male,weather,maxtemp,RH,maxwind
1108,0,13,87,0
1459,0,14,94,45
1099,0,14,82,55
1651,1,13,78,0
2117,0,16,81,37
2223,0,16,73,0


young,weather,maxtemp,RH,maxwind
1122,0,13,87,0
1239,0,14,94,45
1059,0,14,82,55
1606,1,13,78,0
2318,0,16,81,37
2289,0,16,73,0


adult,weather,maxtemp,RH,maxwind
1210,0,13,87,0
1562,0,14,94,45
1316,0,14,82,55
1841,1,13,78,0
2505,0,16,81,37
2689,0,16,73,0


## Index Holdout (Based on selected week)

In [3]:
allIndex = c(1:nrow(ts1))
semanaEsc = c(101:107)
tr = allIndex[-semanaEsc]
ts = semanaEsc

cat("TR")
tr
cat("TS")
ts

TR

[1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 108 109 110 111 112 113 114 115
[109] 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
[127] 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
[145] 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169
[163] 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
[181] 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
[199] 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
[217] 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241
[235] 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257

TS

[1] 101 102 103 104 105 106 107

## Hybird Model (Holtwinters + LM)

In [9]:
HybridModel = function(ts1,ts2,ts3,ts4,ts5,firstDay,lastDay){
    # Create Dataframe for Prediction Storage
    preds <- data.frame(matrix(ncol = 8, nrow = 0))
    # Name the Columns
    colnames(preds) <- c('ts','v1','v2','v3','v4','v5','v6','v7')
    
    # Holdout based on selected week
    allIndex = c(1:nrow(ts1))
    semanaEsc = c(firstDay:lastDay) #101-107
    tr = allIndex[-semanaEsc]
    ts = semanaEsc

    cat("TR")
    tr
    cat("TS")
    ts


    K=7
    Test=K # H, the number of multi-ahead steps, adjust if needed
    S=K # step jump: set in this case to 7 predictions
    timeSeries = c("all","female","male","young","adult")

    for (t in 1:length(timeSeries)){

        switch(  
        t,  
        "all"= {data=ts1},
        "female"= {data=ts2},
        "male"= {data=ts3},
        "young"= {data=ts4},
        "adult"= {data=ts5},
        ) 
        cat("TS:",timeSeries[t],"\n")

        d1 = data[,1] # 1ª coluna
        L = length(d1) # 257

        # rminer:
        timelags=c(1:7) # 1 previous day until 7 previous days
        D=CasesSeries(d1,timelags) # note: nrow(D) is smaller by max timelags than length(d1)

        YR=diff(range(d1)) # global Y range, use the same range for the NMAE calculation in all iterations


        dtr = ts(d1[tr],frequency=K)
        M = suppressWarnings(HoltWinters(dtr)) 
        PrevPred = M$fitted[1:nrow(M$fitted)]
        Pred = forecast(M,h=length(ts))$mean[1:Test]

        # Creating a Dataframe with all univariate predictions
        uniPred = c(PrevPred,Pred)
        HD = cbind(uniPred=uniPred,ts1[1:length(uniPred),2:5],y=d1[1:length(uniPred)])
        HD = data.frame(HD)

        TRSIZE=length(PrevPred)
        LPRED=length(Pred)
        RSIZE=TRSIZE+LPRED

        # Creating Multivariate Model with new Dataframe
        M2=fit(y~.,HD[1:TRSIZE,],model="lm") # create forecasting model
        Pred2=predict(M2,HD[(TRSIZE+1):(RSIZE),]) # multi-step ahead forecasts
        mae=mmetric(y=d1[ts],x=Pred2,metric="MAE",val=YR)
        nmae=mmetric(y=d1[ts],x=Pred2,metric="NMAE",val=YR)

        cat("Predictions:",Pred2,"\n")
        cat("MAE:",mae,"\n")
        cat("NMAE:",nmae,"\n")

        preds[nrow(preds) + 1,] = c(timeSeries[t],Pred2[1],Pred2[2],Pred2[3],Pred2[4],Pred2[5],Pred2[6],Pred2[7])     

    }
    preds
}

In [10]:
HybridModel(ts1,ts2,ts3,ts4,ts5,101,107)

TRTSTS: all 
Predictions: 4793.177 3362.258 3557.21 3618.383 4036.207 4166.177 5026.176 
MAE: 682.2695 
NMAE: 4.441858 
TS: female 
Predictions: 2193.804 1458.586 1539.796 1626.258 1851.835 1931.906 2501.973 
MAE: 393.2683 
NMAE: 5.278061 
TS: male 
Predictions: 2309.879 1706.693 1768.507 1802.096 1879.817 2060.197 2290.446 
MAE: 260.487 
NMAE: 3.788901 
TS: young 
Predictions: 2432.425 1609.163 1697.888 1772.781 2009.02 2154.614 2889.061 
MAE: 492.184 
NMAE: 6.738554 
TS: adult 
Predictions: 2366.276 1631.468 1720.03 1750.448 1954.509 2031.079 2343.703 
MAE: 351.7777 
NMAE: 4.366655 


ts,v1,v2,v3,v4,v5,v6,v7
all,4793.17726824164,3362.25808068743,3557.20997498152,3618.38284728205,4036.20671230761,4166.17709964904,5026.17602233892
female,2193.8036766984,1458.58647770924,1539.79593281009,1626.25799196726,1851.83547726247,1931.90604128389,2501.97323126097
male,2309.87906053873,1706.6926624766,1768.50729898158,1802.09598822059,1879.81746053277,2060.19700100991,2290.44610095395
young,2432.42491495824,1609.16295021767,1697.88812860615,1772.78050459761,2009.0204388966,2154.6136472318,2889.06050587915
adult,2366.27550703726,1631.46817606711,1720.03000756004,1750.4477992549,1954.50863637271,2031.07947115874,2343.70282730685


## Optimization (Hill Climbing)

In [8]:
hillClimbing(preds)

i: 1 s: 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 1 f: 440.19 s' 0 1 0 1 0 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 0 0 1 f: 489.21 
i: 50 s: 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 f: 1077.71 s' 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 f: 1218.71 
i: 100 s: 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 f: 1269.84 s' 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 f: 1155.3 
i: 150 s: 0 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1 1 f: 1430.14 s' 0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 f: 1161.73 
i: 200 s: 0 0 0 1 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 1 1 f: 1430.14 s' 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 f: 1182.84 
i: 250 s: 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 f: 1547.89 s' 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 

In [7]:
hillClimbing = function(preds){
    # Optimization(Hill Climbing)
    source("./otimizacao/hill.R") #  hclimbing is defined here

    #Create dimension of time series
    all = unlist(preds[1,])[-1]
    female = unlist(preds[2,])[-1]
    male = unlist(preds[3,])[-1]
    young = unlist(preds[4,])[-1]
    adult = unlist(preds[5,])[-1]

    pred=as.numeric((c(all, female, male, young, adult)))
    #pred=(c(all=all, female=female, male=male, young=young, adult=adult))
    #pred=c(4974,3228,3191,4153,4307,4660,6193,2299,1442,1427,2035,2043,2207,2894,2390,1606,1627,1880,2028,2227,2967,2680,1625,1688,2208,2282,2441,3115,2294,1603,1503,1945,2025,2219,3078)


    # variables
    D=35 #dimension
    N=7 #days of the week
    custo=c(rep(350,N),rep(150,N),rep(100,N),rep(100,N),rep(120,N))
    solution=solution=sample(c(0,1), replace=TRUE, size=D)


    # evaluation function:
    #eval=function(x) profit(x)

    profit=function(x) 
    { 
    vendas=sales(pred)
    p=sum(x*(vendas-custo))
    return(p)
    }

    sales= function(x)
    {
    vector=c()
    for (i in 1:length(x)) {
    if(i<36){
    if(x[i]<800) sale=0.08*x[i] else sale=0.12*x[i]
    }
    if(i<29){
    if(x[i]<3000) sale=0.04*x[i] else sale=0.05*x[i]
    }
    if(i<22){
    if(x[i]<1800) sale=0.04*x[i] else sale=0.07*x[i]
    }
    if(i<15){
    if(x[i]<1800) sale=0.08*x[i] else sale=0.13*x[i]
    }
    if(i<8){
    if(x[i]<5000) sale=0.06*x[i] else sale=0.09*x[i]
    }
    vector <- c(vector, sale)
    }
    return(vector)
    }

    # hill climbing search
    N=1000 # 1000 searches
    REPORT=N/20 # report results
    lower=rep(0,D) # lower bounds
    upper=rep(1,D) #  upper bounds


    # slight change of a real par under a normal u(0,0.5) function:
    rchange1=function(par,lower,upper) # change for hclimbing
    { hchange(par,lower=lower,upper=upper,rnorm,mean=0,sd=0.5,round=TRUE) }

    HC=hclimbing(par=solution,fn=profit,change=rchange1,lower=lower,upper=upper,type="max",
           control=list(maxit=N,REPORT=REPORT,digits=2))
    cat("best solution:",HC$sol,"evaluation function",HC$eval,"\n")
}

In [23]:
# ciclo que corra todos os algoritmos de otimizacao